In [ ]:
"""Real time plotting of current using kivy
"""
# %matplotlib inline

from kivy.lang import Builder
from kivy.app import App
from kivy.uix.gridlayout import GridLayout
from kivy.garden.graph import MeshLinePlot, LinePlot, Graph
from kivy.clock import Clock
from threading import Thread
import visa
import numpy as np
import time
from scipy.stats import linregress
import os.path # for checking file path 
from kivy.uix.popup import Popup
from kivy.uix.label import Label
from kivy.uix.button import Button
from kivy.uix.textinput import TextInput
from kivy.uix.boxlayout import BoxLayout
#import sys
#from multiprocessing import Process


rm = visa.ResourceManager('C:/Windows/System32/visa64.dll') # give exact location of the driver- only necessary in jupyter 
rm.list_resources()
kly = rm.open_resource('GPIB0::22::INSTR')

kly.write("*rst; status:preset; *cls") # return kly to GPIB defaults

global marker, reverse, markerIII

marker = 0
markerIII = 0

def yes(self):
    global file_name, popup, f, markerIII
    markerIII = 2
    file_name = os.path.join(save_path, file_name)
    f = open(file_name,"w+")
    popup.dismiss()
    return 



def no(self):
    global markerIII
    markerIII = 1
    
    def turn_off(self):
        global popupII, textinput, f, markerIII
        markerIII = 2
        file_name = os.path.join(save_path, textinput.text)
        f = open(file_name,"w+")
        popupII.dismiss()
        return
    
    
    
    global popup, popupII, file_name, textinput
    popup.dismiss()
    
    boxIII = BoxLayout(orientation='vertical')
    boxIII.add_widget(Label(text = 'Enter new filename: '))
    textinput = TextInput(multiline=False)
    boxIII.add_widget(textinput)
    
    popupII = Popup(title='New Filename', content=boxIII, size_hint=(None, None), size=(400, 200),auto_dismiss=False)
    popupII.open()

    textinput.bind(on_text_validate=turn_off)
    
    

    return

def get_microphone_level():
    """
    source: http://stackoverflow.com/questions/26478315/getting-volume-levels-from-pyaudio-for-use-in-arduino
    audioop.max alternative to audioop.rms
    """
    
    global levels, levelsr, pause, endV, popup, f, markerIII
    
    
    

    if os.path.isfile(file_name) == True:
        box = BoxLayout(orientation='vertical')
        boxII = BoxLayout(orientation='horizontal', padding=[10,10], spacing=[20,20])
        btn1 = Button(text='Yes')
        boxII.add_widget(btn1)
        btn2 = Button(text='No')
        boxII.add_widget(btn2)
        box.add_widget(Label(text='Do you want to reuse the same filename?'))
        box.add_widget(boxII)
        

        popup = Popup(title='Filename Overwrite', content=box, size_hint=(None, None), size=(400, 200),auto_dismiss=False)

        btn1.bind(on_press=yes)
        btn2.bind(on_press=no)
        popup.open()
        
        btn1.bind(on_press=yes)
        btn2.bind(on_press=no)
        while markerIII != 2:
            time.sleep(0.001)

    else:
        f = open(file_name,"w+")
    
    
    kly.write("*RST") # return 6487 to GPIB defaults
    kly.write("SOUR:VOLT:RANG %i" %10) # select 10V source range
    kly.write("SOUR:VOLT %i" %0) # Setvoltage source output to 0
    kly.write("SOUR:VOLT:ILIM 20E-3") # Set current limit to 20 mA
    kly.write("SOUR:VOLT:STAT ON") # Put voltage source in operate

    kly.write("FUNC 'CURR'") # select current function
    kly.write("SYST:ZCH ON") # enable zero check
    kly.write("CURR:RANG 20e-3") # select 20 mA range
    kly.write("INIT") # Trigger reading to be used as zero correction
    string0 = kly.query("READ?") # for some reaon zero check kept varying without this step??


    kly.write("SYST:ZCOR:STAT OFF") # turn zero correct off
    kly.write("SYST:ZCOR:ACQ") # use last reading taken as zero correct value
    kly.write("SYST:ZCOR ON") # perform zero correction
    kly.write("CURR:RANG:AUTO ON") # enable auto range
    kly.write("SYST:ZCH OFF") # disable zero check


    num_steps = ((endV-startV)/interval)+1


    plot_array = np.array([])


    voltages = np.linspace(startV, endV, num_steps)
    voltagesr = np.linspace(endV, startV, num_steps)



    for j in range(0,cycles):

        k = 0
        u = 21
        current_tot = np.zeros(int(num_steps))
        voltage_tot = np.zeros(int(num_steps))

        for i in voltages:
            

            if markerII == 1:
                kly.write("SOUR:VOLT %i" %0) # Setvoltage source output to 0
                f.close() 
                return 
            
            kly.write("SOUR:VOLT %f" %i) # Setvoltage source output to i
            voltage = i
            kly.write("SOUR:VOLT:STAT ON") # Put voltage source in operate
            kly.write("CURR:RANG:AUTO ON") # enable auto range
            time.sleep(pause) # Delays next command for number of seconds entered for pause
            string = kly.query("READ?") # trigger and return one reading
            strdata = string.replace("A", "") # remove the units from the string- Amps
            shortstrdata = strdata[:13] # shorten the string to include only the current reading value
            current = float(shortstrdata) # convert from string to float
            f.write("%f    %E\n" % (voltage, current))
            print(voltage,current)
            current_tot[k] = current_tot[k] + current
            voltage_tot[k] = voltage_tot[k] + voltage

            
            magnitude = int(np.log10(np.abs(float(current_tot[0])))) # get order of magnitude
            levels = (current_tot*(10**(abs(magnitude)+1)))
            
            

            k = k+1

        f.write("\n")
        
        current_totr = np.zeros(int(num_steps+u))
        voltage_totr = np.zeros(int(num_steps))
        
    
        if reverse == 'y':
            

                levels = np.pad(levels, (0, 21), 'constant')
                for j in voltagesr:
                    
                    if markerII == 1:
                        kly.write("SOUR:VOLT %i" %0) # Setvoltage source output to 0
                        f.close() 
                        return
                    
                    kly.write("SOUR:VOLT %f" %j) # Setvoltage source output to j
                    voltage = j
                    kly.write("SOUR:VOLT:STAT ON") # Put voltage source in operate
                    kly.write("CURR:RANG:AUTO ON") # enable auto range
                    time.sleep(pause) # Delays next command for number of seconds entered for pause
                    string = kly.query("READ?") # trigger and return one reading
                    strdata = string.replace("A", "") # remove the units from the string- Amps
                    shortstrdata = strdata[:13] # shorten the string to include only the current reading value
                    current = float(shortstrdata) # convert from string to float
                    f.write("%f    %E\n" % (voltage, current))
                    current_totr[u] = current_totr[u] + current
                    #voltage_totr[u] = voltage_totr[u] + voltage
                    global marker
                    marker = 1
                    magnitude = int(np.log10(np.abs(float(current_tot[0]))))
                    levelsr = levels+ (current_totr*(10**(abs(magnitude)+1)))
                    
                    u = u+1

        f.write("\n")

        kly.write("SYST:LOC") # Put in local mode to avoid 800 illegal error when next command is run
        kly.write("TRAC:CLE") # clear trace to get ready for the next measurement 
        kly.write("SYST:REM")
    
    f.close() 
    
        
   

class Logic(GridLayout):
    
    def __init__(self,):
        super(Logic, self).__init__()
        self.plot = MeshLinePlot(mode='line_strip',color=[1, 0, 0, 1])
        self.plotII = MeshLinePlot(mode='line_strip',color=[1, 1, 0, 1])
    
    def start(self):
        #self.ids.graph = Graph(ymin=-0.1, ymax=0.1)
        self.ids.graph.add_plot(self.plot)

        if reverse == 'y':
            self.ids.graph.add_plot(self.plotII)
        
        Clock.schedule_interval(self.get_value, pause)
        
    global markerII
    markerII = 0
    
    def stop(self):
        global markerII
        markerII = 1
        Clock.unschedule(self.get_value)
        
        #get_microphone_level.stop()
        #sys.exit()

    def get_value(self, dt):

        if marker ==0: 
            self.plot.points = [((i*5), ((j+([(i*5, abs(j)) for i, j in enumerate(levels)][0][1]))*7)/endV) for i, j in enumerate(levels)]
        else:
            self.plot.points = [((i*5), ((j+([(i*5, abs(j)) for i, j in enumerate(levels)][0][1]))*7)/endV) for i, j in enumerate(levels)]
            self.plotII.points = [((i*5), ((j+([(i*5, abs(j)) for i, j in enumerate(levelsr)][0][1]))*7)/endV) for i, j in enumerate(levelsr[::-1])]
            
            
    def start_measurement(self):
        
        
        """FIX HARD CODED SAVE PATH, REVERSE< and FILENAME"""
        global startV, endV, interval, pause, cycles, reverse, file_name, save_path

        #USER INPUTS
        file_name = self.ids.file_name.text
        save_path = self.ids.curr_folder.text
        startV = np.float(self.ids.startV.text) # V
        endV = np.float(self.ids.endV.text) # V
        interval=np.float(self.ids.interval.text) # V
        pause = np.float(self.ids.pause.text) # seconds
        cycles = np.int(self.ids.scan_number.text) # number of cycles
        reverse = self.ids.cycle.text # 'y' for reverse scans in addition to forward, 'n' for no reverse scans
        file_name = os.path.join(save_path, file_name)

        global get_level_thread
        get_level_thread = Thread(target = get_microphone_level)
        get_level_thread.daemon = True
        get_level_thread.start()
        #get_level_thread = Process(target = get_microphone_level)
        #get_level_thread.start()


class RealTimeMicrophone(App):
    def build(self):
        return Builder.load_file("lookII.kv")

if __name__ == "__main__":
    levels = []  # store levels of microphone
    levelsr = []
    RealTimeMicrophone().run()